In [59]:
import pandas as pd
import twitter
from twitter_accounts import accounts
from geotext import GeoText
from pygeocoder import Geocoder

In [23]:
app = accounts["social"]

auth = twitter.oauth.OAuth(
    app["token"],
    app["token_secret"],
    app["api_key"],
    app["api_secret"])

twitter_api = twitter.Twitter(auth=auth)

In [3]:
query = "pharo"
count = 200

search_results = twitter_api.search.tweets(q=query, count=count)

statuses = search_results["statuses"]

In [26]:
locations = [tweet['user']['location'] for tweet in statuses]
locations[:5]

['Tokyo', '', '', 'www.soundcloud.com/cight', '睡眠の重要性 / 神奈川県(Japan)']

In [52]:
cities = [GeoText(loc).cities for loc in locations]
countries = [GeoText(loc).countries for loc in locations]

In [53]:
def first_or_none(arr, i):
    if arr[i] == []:
        arr[i] = None
    else:
        arr[i] = arr[i][0]

In [54]:
for i in range(len(locations)):
    first_or_none(cities, i)
    first_or_none(countries, i)

In [76]:
places = pd.DataFrame({'city': cities, 'country': countries})
places = places.dropna(how='all')
places.head()

,city,country
0,Tokyo,None
4,None,Japan
5,Tokyo,Japan
9,Marseille,France
10,Marseille,France


In [91]:
places['lat'] = None
places['lon'] = None

for index, row in places.iterrows():
    if row['city'] != None:
        location = row['city']
    else:
        location = row['country']
        
    geocode = Geocoder.geocode(location)
    coord = geocode.coordinates
    
    row['lat'] = coord[0]
    row['lon'] = coord[1]

In [92]:
places.head()

,city,country,lat,lon
0,Tokyo,None,35.6895,139.692
4,None,Japan,36.2048,138.253
5,Tokyo,Japan,35.6895,139.692
9,Marseille,France,43.2965,5.36978
10,Marseille,France,43.2965,5.36978
